In [14]:
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [15]:
load_dotenv()

llm= ChatOpenAI()

In [16]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explaination: str

In [17]:
def generate_joke(state:JokeState):
    prompt= f"Wriite a funny joke on following topic: {state['topic']}"

    res= llm.invoke(prompt).content

    return {'joke':res}

In [18]:
def explain_joke(state:JokeState):
    prompt= f"Explain the following joke in simple manner: {state['joke']}"

    res= llm.invoke(prompt).content

    return {'explaination':res}

In [19]:
graph= StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('explain_joke', explain_joke)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'explain_joke')
graph.add_edge('explain_joke', END)

checkpointer= InMemorySaver()

wf= graph.compile(checkpointer=checkpointer)

In [20]:
thread_id="1"
config= {'configurable':{'thread_id': thread_id}}

wf.invoke({'topic':'pizza'}, config=config)

{'topic': 'pizza',
 'joke': 'Why did the slice of pizza go to the party? Because it knew it would be a "pizza" cake!',
 'explaination': 'This joke is a play on words. The phrase "pizza cake" sounds like "piece of cake," which means something is easy or simple. In this joke, the slice of pizza went to the party because it knew it would be easy or simple for it to have fun and enjoy itself there.'}

In [21]:
wf.get_state(config=config)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the slice of pizza go to the party? Because it knew it would be a "pizza" cake!', 'explaination': 'This joke is a play on words. The phrase "pizza cake" sounds like "piece of cake," which means something is easy or simple. In this joke, the slice of pizza went to the party because it knew it would be easy or simple for it to have fun and enjoy itself there.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f071f5c-ebee-654a-8002-35cc9780ede0'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-05T12:14:59.802861+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f071f5c-df6b-6774-8001-5b62c1e0cdbe'}}, tasks=(), interrupts=())

In [22]:
list(wf.get_state_history(config))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the slice of pizza go to the party? Because it knew it would be a "pizza" cake!', 'explaination': 'This joke is a play on words. The phrase "pizza cake" sounds like "piece of cake," which means something is easy or simple. In this joke, the slice of pizza went to the party because it knew it would be easy or simple for it to have fun and enjoy itself there.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f071f5c-ebee-654a-8002-35cc9780ede0'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-05T12:14:59.802861+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f071f5c-df6b-6774-8001-5b62c1e0cdbe'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the slice of pizza go to the party? Because it knew it would be a "pizza" cake!'}, next=('explain_joke',), config={'configurable': 

## Time Travel

In [23]:
wf.get_state({'configurable':{'thread_id':thread_id, 'checkpoint_id':'1f07161b-d422-60e0-8000-6b0ce95d0a6e'}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f07161b-d422-60e0-8000-6b0ce95d0a6e'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [24]:
wf.update_state({'configurable':{'thread_id':thread_id, 'checkpoint_id':'1f07161b-d422-60e0-8000-6b0ce95d0a6e',"checkpoint_ns":''}},{'topic':'pencil'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f071f5c-ecbe-66c0-8000-b93d3c80f213'}}

In [25]:
list(wf.get_state_history(config))

[StateSnapshot(values={'topic': 'pencil'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f071f5c-ecbe-66c0-8000-b93d3c80f213'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-08-05T12:14:59.888095+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07161b-d422-60e0-8000-6b0ce95d0a6e'}}, tasks=(PregelTask(id='a5722b16-bd55-1902-fa1d-c91d5192b020', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the slice of pizza go to the party? Because it knew it would be a "pizza" cake!', 'explaination': 'This joke is a play on words. The phrase "pizza cake" sounds like "piece of cake," which means something is easy or simple. In this joke, the slice of pizza went to the party because it knew it would be easy or simple for i

In [26]:
list(wf.get_state_history(config))

[StateSnapshot(values={'topic': 'pencil'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f071f5c-ecbe-66c0-8000-b93d3c80f213'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-08-05T12:14:59.888095+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07161b-d422-60e0-8000-6b0ce95d0a6e'}}, tasks=(PregelTask(id='a5722b16-bd55-1902-fa1d-c91d5192b020', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the slice of pizza go to the party? Because it knew it would be a "pizza" cake!', 'explaination': 'This joke is a play on words. The phrase "pizza cake" sounds like "piece of cake," which means something is easy or simple. In this joke, the slice of pizza went to the party because it knew it would be easy or simple for i